In [1]:
import numpy as np
from random import shuffle
from sys import exit
from copy import deepcopy
from warnings import warn

In [16]:
card_per_player = {2:7, 3:6, 4:6, 5:6}
players = 2
card_per_turn = 2
danger_thresh = 4   #how much more a player can ruin another pile with respect to the preferred one because of other players' preference
one_more_tresh = 2   #how much a player can ruin a pile during a non-obligatory move
verbose="vvv"
N = players
n = 2 #card_per_turn
m = card_per_player[players]

In [17]:
def pick_one(hand):
    global n,deck_empty
    if deck == [] and not deck_empty:
        n = card_per_turn - 1
        print("\n------------------------------- Deck is over, last few rounds. Hold on! -------------------------------\n")
        deck_empty = True
    if deck == []:
        return hand
    hand.append(deck[0])
    deck.pop(0)
    return hand

In [18]:
def scan_hand_(hand,discards=[],obligatory_move=True):
# delta is the distance of the bast match
# ind contains first the in hand index then the pile index
    tops = [pile[-1] for pile in piles]
    delta = 100
    ind = []
    for i, hand_i in enumerate(hand):
        for j, top_j in enumerate(tops):
            if j in discards: continue
            old_delta = delta
            old_ind = ind
            delta = hand_i - top_j
            if j > 1: delta = -delta
            ind = [i,j]
            if delta == -10:
                return delta, ind
            elif delta >= old_delta or delta < 0 :
                delta = old_delta
                ind = old_ind
    if delta==100 and discards==[] and obligatory_move: end_game_and_recap() #replace this with exit program command when the code is implemented in a script
    return delta, ind

In [ ]:
def scan_inside_hand_combo(hand):
    #example: hand=[40,41,50,51,70,1,2,3,20,30,31]
    hand_=np.asarray(hand)
    sort_indices=hand_.argsort()
    hand_ = np.sort(hand_)
    #hand_ = [1,2,3,20,30,31,40,41,50,51,70]
    
    pairs=[]
    n_card=len(hand)
    for i in range(n_card):
        card_i=hand_[i]
        for j in range(i+1,n_card):
            card_j=hand_[j]
            if card_j-card_i==10:
                pairs.append([i,j])
    #pairs = [[3, 4], [4, 6], [5, 7], [6, 8], [7, 9]]
    
    combo_list_idx = []
    for (i,j) in pairs:
        # print('')
        # print('i,j: ',i,j)
        # print('ccombo_list_idx: ',combo_list_idx)
        ij_in_a_combo=False
        for l,combo_l in enumerate(combo_list_idx):
            if i in combo_l and not j in combo_l:
                combo_list_idx[l].append(j)
                ij_in_a_combo = True
                break
            elif i not in combo_l and j in combo_l:
                combo_list_idx[l].append(i)
                ij_in_a_combo = True
                break
            elif i in combo_l and j in combo_l:
                ij_in_a_combo = True            
                break
            else:
                ij_in_a_combo = False
        if not ij_in_a_combo: combo_list_idx.append([i,j])
        # print('ij_in_a_combo',ij_in_a_combo)
        # print('combo_list_idx: ',combo_list_idx)
        # print('')
    # combo_list_idx = [[3, 4, 6, 8], [5, 7, 9]]
    
    combo_list_idx_list_original_order = []
    for i,list_i in enumerate(combo_list_idx):
        new_list = []
        for j,element_j in enumerate(list_i):
            new_list.append(sort_indices[element_j])
        combo_list_idx_list_original_order.append(new_list)
    # combo_list_idx_list_original_order=[[8, 9, 0, 2], [10, 1, 3]]
    
    combo_list_cards = [] 
    for i,list_i in enumerate(combo_list_idx_list_original_order):
        new_list = []
        for j,element_j in enumerate(list_i):
            new_list.append(hand[element_j])
        combo_list_cards.append(new_list)
    
    #output:
    # [20, 30, 40, 50]
    # [31, 41, 51]

    #if not combo_list_cards == []: print("Found Combo: ",combo_list_cards)
    return combo_list_idx_list_original_order,combo_list_cards

In [20]:
def scan_hand(hand,discards=[],obligatory_move=True):
    delta,ind = scan_hand_(hand,discards=discards,obligatory_move=obligatory_move)
    if ind==[]:
        return delta,[None,None]
    elif len(ind)==2:
        return delta,ind
    else:
        raise ValueError("Output of scan_hand not recognized")

In [21]:
def print_piles_vertical(piles):
    # Find the maximum length among all lists
    max_len = max(len(pile) for pile in piles)
    
    # Print the piles in columns
    for i in range(max_len):
        row = []
        for pile in piles:
            # If the pile still has elements at this index, add the element
            if i < len(pile):
                row.append(str(pile[i]))
            else:
                row.append("")  # Empty space if the pile is finished
        
        # Print the row with uniform spacing
        print(f"{row[0]:<4} {row[1]:<4} {row[2]:<4} {row[3]:<4}")

In [22]:
def print_piles(piles):

    print("")
    print("Printing Piles")
    for pile in piles:
        string = ""
        for element in pile:
             string = string + f"{element:3d}" + " "
        print(string)
    print("")

In [23]:
def end_game_and_recap():
    #print_piles(piles)
    raise ValueError("Game Over")

In [24]:
#interaction(hands[inactive_players])
def interaction(hands):
    #things to implement: give higher priority to closest players
    n_hands = len(hands)
    delta_list = []
    pile_idx_list = []
    for i in range(n_hands):
        delta,(card_idx,pile_idx)=scan_hand(hands[i],obligatory_move=False)
        pile_idx_list.append(pile_idx)
    return list(set(pile_idx_list))

In [25]:
def place_card(hand,card_idx,pile_idx):
    print('Card placed: ', hand[card_idx], 'on pile ',pile_idx)
    piles[pile_idx].append(hand[card_idx]) #add played card to the piles
    hand.pop(card_idx) #remove card played from the hand
    return hand

In [26]:
def play_card(hand,player_idx,obligatory_move=True):
    
    delta_1,(card_idx1,pile_idx1)=scan_hand(hand,obligatory_move=obligatory_move)
    
    if delta_1==-10:
        hand=place_card(hand,card_idx1,pile_idx1)
        return hand
        
    combo_list_idx,combo_list_cards = scan_inside_hand_combo(hand)
    if len(combo_list_cards)==1:
        combo_list_cards = combo_list_cards[0]
        combo_list_idx = combo_list_idx[0]
        if card_idx1 in combo_list_idx:
            print('Found combo: ',combo_list_cards)
            if pile_idx1 in [0,1]: #ascending
                card = np.asarray(combo_list_cards).max()
                card_idx1 = hand.index(card)
                delta_1 = np.asarray(combo_list_cards).min()-piles[pile_idx1][-1]
            elif pile_idx1 in [2,3]: #descending
                card = np.asarray(combo_list_cards).min()
                card_idx1 = hand.index(card)
                delta_1 = np.asarray(combo_list_cards).max()-piles[pile_idx1][-1]
    elif combo_list_cards==[]:
        pass
    else:
        warn("Multiple combo not implemented, ignoring it!")
        pass

    if not obligatory_move and delta_1>one_more_tresh:  #maybe must be moved below
        return hand   #maybe must be moved below
        
    else:  
        hands_non_active = [hand for i,hand in enumerate(hands) if not i==player_idx]
        dont_pile_idx_list = interaction(hands_non_active)
        
        if not pile_idx1 in dont_pile_idx_list: #case 1: the pile of interest can be played
            hand=place_card(hand,card_idx1,pile_idx1)
            return hand
        elif dont_pile_idx_list==[0,1,2,3]: #case 2: all piles are taken, so fuck it
            hand=place_card(hand,card_idx1,pile_idx1)
            return hand
        else: #case 3: check other piles
            delta_2,(card_idx2,pile_idx2)=scan_hand(hand,discards=dont_pile_idx_list,obligatory_move=obligatory_move)
            if delta_2-delta_1>danger_thresh or delta_2==100:
                hand=place_card(hand,card_idx1,pile_idx1)
                return hand
            else:
                if not obligatory_move and delta_2>one_more_tresh:
                    return hand
                else:
                    print('Piles n.',dont_pile_idx_list, ' are taken, changing move')
                    hand=place_card(hand,card_idx2,pile_idx2)
                    return hand

In [36]:
def play_turn(player_idx):
    n_played=n   #this is incremented only in case one or more non-obbligatory cards are played
    hand=hands[player_idx]

    if hand==[]: return

    print('\n----------------- Player ', player_idx,'-----------------\n')
    print('Hand: ',hand)
    print_piles(piles)
    
    for _ in range(n):
        hand=play_card(hand,player_idx)

    while True:
        hand_old=hand.copy()
        hand=play_card(hand,player_idx,obligatory_move=False)
        #print(hand,hand_old)
        if hand==hand_old: break
        n_played = n_played + 1
        
    if verbose=="vvv": print('n. played cards: ',n_played)
    for _ in range(n_played):
        hand=pick_one(hand)
    hands[player_idx]=hand

In [39]:
original_deck = [int(i) for i in np.arange(2,99)]
shuffle(original_deck)
original_piles = [[1],[1],[100],[100]]

In [40]:
#Convention: decreasing piles start with 100 but for sake of ease
# we consider them at -100 so that they are increasing
# in this way we simply change the sign for cards in hands
deck = deepcopy(original_deck)
piles = deepcopy(original_piles)
hands = np.asarray(deck[:N*m]).reshape([N,m]).tolist()
deck = deck[N*m:]

deck_empty = False
n = 2 #card_per_turn
game_won = False

while True:
    if not game_won:
        for player_idx in range(N):
            play_turn(player_idx)
            if hands==[[] for i in range(N)]: game_won=True
    else: break
print('\nWow, we actually won. Must’ve been pure luck… or the cards felt sorry for us.\n')


----------------- Player  0 -----------------

Hand:  [12, 91, 20, 97, 66, 57, 89]

Printing Piles
  1 
  1 
100 
100 

Piles n. [2]  are taken, changing move
Card placed:  97 on pile  3
Piles n. [3]  are taken, changing move
Card placed:  91 on pile  2
Card placed:  89 on pile  2
n. played cards:  3

----------------- Player  1 -----------------

Hand:  [88, 53, 34, 16, 82, 64, 93]

Printing Piles
  1  12 
  1 
100  91  89 
100  97 

Card placed:  88 on pile  2
Card placed:  93 on pile  3
n. played cards:  2

----------------- Player  0 -----------------

Hand:  [12, 20, 66, 57, 18, 45, 63]

Printing Piles
  1 
  1 
100  91  89  88 
100  97  93 

Piles n. [0]  are taken, changing move
Card placed:  12 on pile  1
Card placed:  18 on pile  1
Card placed:  20 on pile  1
n. played cards:  3

----------------- Player  1 -----------------

Hand:  [53, 34, 16, 82, 64, 23, 2]

Printing Piles
  1 
  1  12  18  20 
100  91  89  88 
100  97  93 

Card placed:  2 on pile  0
Piles n. [1]  are tak

ValueError: Game Over